In [ ]:
import numpy as np
import networkx as nx
import time
import math
import scipy as sp
from scipy.sparse import *
from scipy.sparse.linalg import norm

Let's create two graphs from the list of edges downloaded from the Snap database. 

In [ ]:
G1 = nx.read_edgelist('../data/web-Stanford.txt', create_using=nx.DiGraph(), nodetype=int)

G2 = nx.read_edgelist('../data/web-BerkStan.txt', create_using=nx.DiGraph(), nodetype=int)

### Creating the transition probability matrix

In [ ]:
# square matrix of size n x n, where n is the number of nodes in the graph. The matrix is filled with zeros and the (i,j) element is x if the node i is connected to the node j. Where x is 1/(number of nodes connected to i).

def create_matrix(G):
    n = G.number_of_nodes()
    P = sp.sparse.lil_matrix((n,n))
    for i in G.nodes():
        for j in G[i]: #G[i] is the list of nodes connected to i, it's neighbors
            P[i-1,j-1] = 1/len(G[i])
    return P

To ensure that the random process has a unique stationary distribution and it will not stagnate, the transition matrix P is usually modified to be an irreducible stochastic matrix A (called the Google matrix) as follows

$$ A = \alpha \tilde{P} + (1-\alpha)v e^T$$

Where $\tilde{P}$ is defined as 

$$ \tilde{P} = P + v d^T$$

Where $d \in \mathbb{N}^{n \times 1}$ s a binary vector tracing the indices of dangling web-pages with no hyperlinks, i.e., $d(i ) = 1$ if the `ith` page has no hyperlink, $v \in \mathbb{R}^{n \times 1}$ is a probability vector, $e = [1, 1, . . . , 1]^T$ , and $0 < \alpha < 1$ is the so-called damping factor that represents the probability in the model that the surfer transfer by clicking a hyperlink rather than other ways

In [ ]:
n = G2.number_of_nodes()
P = create_matrix(G2)    

the vector `d` solves the dangling nodes problem

In [ ]:
# define d as a nx1 sparse matrix, where n is the number of nodes in the graph. The vector is filled with d(i) = 1 if the i row of the matrix P is filled with zeros, other wise is 0

# d is the vector of dangling nodes
d = sp.sparse.lil_matrix((n,1))
for i in range(n):
    if P[i].sum() == 0:
        d[i] = 1

The vector v is a probability vector, the sum of its elements bust be one

In [ ]:
# define v as the probability vector of size n x 1, where n is the number of nodes in the graph. The vector is filled with 1/n
# https://en.wikipedia.org/wiki/Probability_vector

v = sp.sparse.lil_matrix((n,1))
for i in range(n):
    v[i] = 1/n  

Now we can compute the transition matrix


In [ ]:
Pt = P + v.dot(d.T)

# Pt is a sparse matrix too

In [ ]:
# e is a nx1 sparse matrix filled with ones
e = sp.sparse.lil_matrix((1,n))
for i in range(n):
    e[0,i] = 1

In [ ]:
# # v*eT is a nxn sparse matrix filled all with 1/n, let's call it B

# B = sp.sparse.lil_matrix((n,n))
# for i in range(n):
#     for j in range(n):
#         B[i,j] = 1/n

# A = alpha*Pt + (1-alpha)*B

## Algorithm 1 Shifted-Power method for PageRank with multiple damping factors:

In [ ]:
# list of alpha values, from 0.85 to 0.99 with step 0.01
a = []
for i in range(85,100):
    a.append(i/100)

tau = 10**-6
max_mv = 100

# this should return mv (the number of iteration needed for the convergence), and two vector called x and r. Where x is the vector of the pagerank and r is the residual vector

def Algorithm1(Pt, v, tau, max_mv, a: list):
    # take time of the performance
    start_time = time.time()


    u = Pt.dot(v) - v 
    mv = 1 # number of iteration
    r = sp.sparse.lil_matrix((n,1)) 
    Res = sp.sparse.lil_matrix((len(a),1))
    x = sp.sparse.lil_matrix((n,1))  

    for i in range(len(a)):
        r = a[i]*(u) 
        normed_r = norm(r)
        Res[i] = normed_r 

        if Res[i] > tau:
            x = r + v 

    print("STARTING THE WHILE LOOP\n")

    # take the maximum value of the sparse matrix Res


    while max(Res) > tau and mv < max_mv:
        u = Pt*u # should it be the same u of the beginning?
        mv += 1 
        print("mv = ", mv)
        print("max(Res) = ", max(Res))

        for i in range(len(a)):
            if Res[i] >= tau: 
                r = (a[i]**(mv+1))*(u)
                Res[i] = norm(r)

                if Res[i] > tau:
                    x = r + x

    print("\nEND OF THE WHILE LOOP\n")

    if mv == max_mv:
        print("The algorithm didn't converge in ", max_mv, " iterations")
    else:
        print("The algorithm converged in ", mv, " iterations")

    print("\nThe execution time is %s seconds" % (time.time() - start_time))
       
    return mv, x, r  

In [ ]:
mv, x, r = Algorithm1(Pt, v, tau, max_mv, a)

### Algorithm 2 Arnoldi process

In [ ]:
def Algorithm2():
    pass

## Algorithm 4 shifted-GMRES method for PageRank with multiple damping factors: 

In [ ]:
def Algorithm4(Pt, v, m , a: list, tau , maxit, x: list):
    iter = 1
   
    e1 = sp.sparse.lil_matrix((len(a),1))
    e1[0] = 1

    # identity matrix sparse
    I = sp.sparse.lil_matrix((len(a),len(a)))
    for i in range(n):
        I[i,i] = 1

    # create the page rank vector x
    x = sp.sparse.lil_matrix((n,1))
    for i in range(n):
        x[i] = 1/n

    # create the vector r 
    r = sp.sparse.lil_matrix((len(a),1))
    for i in range(len(a)):
        r[i] = ((1-a[i])/a[i]).dot(v) - ((1/a[i]).dot(I) - Pt).dot(x[i]).dot(e1)
    
    # create the vector Res
    Res = sp.sparse.lil_matrix((len(a),1))
    for i in range(len(a)):
        Res[i] = a[i] * np.linalg.norm(r[i])

    mv = 0

    while max(Res) > tau and mv < maxit:
        # find the k that satisfy the condition res[k] = max(res[i])
        k = 0
        for i in range(len(a)):
            if Res[i] == max(Res):
                k = i
                break
        
        # compute a new vector called delta where delta[i] = (res[i]*a[k])/(res[k]*a[i])
        delta = sp.sparse.lil_matrix((len(a),1))
        for i in range(len(a)) and i != k:
            delta[i] = (Res[i]*a[k])/(Res[k]*a[i])

        # run algorithm 2
        # TO DO

        # j depends on the algorithm 2
        mv = mv + j

        
        # .............

        
        